## Idiom Vocab Helper 

This notebook creates a list of all the know idioms from the given MAGPIE dataset and creates Single-Tokens for them. This list of single-tokens will be inserted into the vocabulary of a LM using **model_download_add_tokensipynb**. 

In [1]:
#TODO: Should I create a list of idioms from the entire data(combined training, dev & test sets)? Would it still be 
# called zero-shot if I do so?

In [50]:
import pandas as pd
pd.options.display.max_colwidth=500
import numpy as np

In [46]:
root_dir = '../'
base_dir = root_dir + 'data/magpie/'
data_file = base_dir + 'MAGPIE_filtered_split_typebased.jsonl'

output_token_dir = root_dir + 'data/token_files/'

In [4]:
df_data = pd.read_json(data_file, lines=True)
df_data

,confidence,context,document_id,genre,id,idiom,judgment_count,label,label_distribution,non_standard_usage_explanations,offsets,sentence_no,split,variant_type
0,1.000000,"[Please , can we close the doggy postbag for now !, Remember that RUNNING is looking for all kinds of safety tips ., For example , with fell running and mountain marathons gaining in popularity , how about some ideas for safe running off the beaten track ?, FITNESS CLINIC, DIET]",AR7,W pop lore,0,off the beaten track,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': 0.0}",[],"[[117, 120], [125, 131], [132, 137]]",626,training,identical
1,0.770109,"[But it 's a selfish family , I 'd say ., They take what they want ., I 'd keep him well in the running ., Then of course there 's Desmond 's wife — I 'd forgotten her ., I did n't get much of an impression of her .]",H9D,W fict prose,1,in the running,10,i,"{'?': 0.0, 'f': 0.0, 'i': 0.770109357701371, 'l': 0.22989064229862802, 'o': 0.0}",[],"[[19, 21], [26, 33]]",1520,training,identical
2,1.000000,"[And I looked behind , and he was just sitting there staring like that ., Oh my god ., He gives me the creeps , so I looked round , hmm hmm ., I mean , what is she doing ?, What does she want ?]",KNY,S conv,2,give someone the creeps,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': 0.0}",[],"[[3, 8], [9, 11], [16, 22]]",1850,training,combined-inflection
3,1.000000,"[Especially this year ., Makes me hair stand on end just thinking about it.’, ‘ He 's done us proud , as well,’ says Granville ., ‘ He had the chance to go to the States with them but he said , ‘ No’ ., Other commitments.’]",CK4,W pop lore,3,do someone proud,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': 0.0}",[],"[[8, 12], [13, 15], [16, 21]]",613,training,combined-inflection
4,1.000000,"[Rather , what is needed most is a new way of thinking – new “ software ” ( though effective “ hard ” green technologies also are essential ) ., As we saw in the postcommunist world , changing attitudes is often the hardest problem of all ., People quickly embraced formal democracy , but the tolerance and compromise that is at the heart of the democratic process took time to take root ., , ]",p63d3559,PMB,4,take root,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': 0.0}",[],"[[136, 140], [141, 145]]",15,training,identical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48390,0.854973,"[The running of multiple sessions has been another means of saving which , though not always a problem , has led to truncated teaching sessions , late - coming and absenteeism on the part of teachers and pupils ., Teacher absenteeism is also exacerbated by general shortages of goods and services : teachers , like other employees , can spend half their days chasing scarce consumer goods for their families , instead of being in the classroom ., Many also have second or third jobs to make ends ...",B12,W ac:polit law edu,48390,make ends meet,9,i,"{'?': 0.0, 'f': 0.0, 'i': 0.8549734944978761, 'l': 0.145026505502123, 'o': 0.0}",[],"[[39, 43], [44, 48], [49, 53]]",246,test,identical
48391,1.000000,"[you understand the process ?, Yeah , yeah, Take people to objections , take them to where you want them to be and bear in mind you 're always looking for an objection, Yeah, right , another thing , we wanna get more quotes , right]",KDJ,S conv,48391,bear in mind,3,i,"{'?': 0.0, 'f': 0.0, 'i': 1.0, 'l': 0.0, 'o': 0.0}",[],"[[71, 75], [76, 78], [79, 83]]",207,training,identical
48392,1.000000,"[The same implications attach to the playing of games or the membership of clubs and so on , although what is of even more interest are the ' unwritten ' rules which underwrite the more formal , quasi - legal , ones ., Without unwritten rules civilised life would be impossible ., Indeed we are rarely aware of them as rules , until they are broken , since they are typical of the settings in which we received our moral training ., Many were originally instinctive and , to that limited extent ,...",CM8,W

In [5]:
# NOTE: Considering the entire data here

In [6]:
df_data['idiom']

0           off the beaten track
1                 in the running
2        give someone the creeps
3               do someone proud
4                      take root
                  ...           
48390             make ends meet
48391               bear in mind
48392                  as a rule
48393         jack of all trades
48394                 fly a kite
Name: idiom, Length: 48395, dtype: object

In [7]:
# Check if the values in 'idiom' column are used exactly as it is in the sentece
df_data['idiom_exact_match'] = df_data.apply(lambda row: row['idiom'] in row['context'][2], axis=1)
df_data['idiom_exact_match'].value_counts()

True     25308
False    23087
Name: idiom_exact_match, dtype: int64

In [8]:
dist = df_data[[ 'label', 'idiom_exact_match']].value_counts()
pd.DataFrame(dist)

0
label idiom_exact_match       
i     True               21124
      False              15204
l     False               7883
      True                4184

In [33]:
import string
trans = str.maketrans('', '', string.punctuation)

def idiom_to_token(idiom_phrase):
    """Process the given idiom phrase and convert into a string token"""
    idiom_phrase = idiom_phrase.translate(trans)
    idiom_phrase = idiom_phrase.lower().replace(' ', '').lstrip().rstrip()
    return 'ID' + idiom_phrase + 'ID'

### Idioms are not matching exactly
How to deal with ~50% of the examples where the value of the column 'idiom' is not matching exactly with that of its usage in the sentence?

**Option-1**
Just convert the values in 'idiom' column to tokens, irrespective of how they are used in the sentence. In other words, this approach will make the LM model to learn only those tokens which have an exact match. This leads to incomplete experiment, because we will capture only 50% of MWEs as single tokens.

**Option-2**
Use the **offsets** column and extract the actual MWE from the sentence. This will capture all possible MWEs in the data, but the number of unique tokens would be very high.

### Option-1

In [25]:
display(df_data['idiom'].value_counts())
df_data['idiom'].unique().shape[0]

in the long run                200
come to terms with             174
with a view to                 173
bear in mind                   172
for the time being             171
                              ... 
king of beasts                   1
fit to be tied                   1
start off on the right foot      1
in apple-pie order               1
ring off the hook                1
Name: idiom, Length: 1738, dtype: int64

1738

In [43]:
df_data['idiom_token'] = df_data['idiom'].map(idiom_to_token)
df_data[['idiom', 'idiom_token']].sample(10)

,idiom,idiom_token
9878,get under someone's skin,IDgetundersomeonesskinID
27685,on paper,IDonpaperID
12768,keep a lid on,IDkeepalidonID
20066,to the tune of,IDtothetuneofID
41993,every last,IDeverylastID
47221,rank and file,IDrankandfileID
32784,turn the screw,IDturnthescrewID
37366,over the counter,IDoverthecounterID
45018,with a vengeance,IDwithavengeanceID
2497,rough and tumble,IDroughandtumbleID


In [62]:
# Save the list of idioms
option_1_idiom_token_file = output_token_dir + 'option1_idiom_tokens.txt'
unique_tokens = df_data['idiom_token'].unique()
np.savetxt(option_1_idiom_token_file, unique_tokens, fmt='%s', header='')
print(f'Saved the {unique_tokens.shape[0]} tokens to {option_1_idiom_token_file}')

Saved the 1738 tokens to ../data/token_files/option1_idiom_tokens.txt


### Option-2

In [55]:
def extract_idiom_phrase(row):
    sentence = row['context'][2]
    # Pair of start and end positions of an MWE within the sentence
    offsets = row['offsets']
    # Being crude here, so that the exact usage of the MWE is captured
    start = offsets[0][0] # Start of the first word
    end = offsets[-1][1] # End of the last word
    return sentence[start:end]
# Extract the MWE from sentences
df_data['idiom_phrase'] = df_data.apply(lambda row: extract_idiom_phrase(row), axis=1)

In [58]:
df_data[['idiom', 'idiom_phrase']].sample(10)

,idiom,idiom_phrase
3823,make waves,making any official waves
31433,do the right thing,doing the right thing
22675,the world and his wife,world and his wife
39148,make tracks,make tracks
38375,down and out,down - and - outs
25452,force someone's hand,force his hand’
13058,in light of,in the light of
21541,above board,above board
4544,hang by a thread,hanging on to life by a thread
46989,slippery slope,slippery slope


In [60]:
df_data['idiom_phrase_token'] = df_data['idiom_phrase'].map(idiom_to_token)
df_data[['idiom_phrase', 'idiom_phrase_token']].sample(10)

,idiom_phrase,idiom_phrase_token
43476,pulled a face,IDpulledafaceID
35659,think twice,IDthinktwiceID
20035,bottom drawer,IDbottomdrawerID
40652,had the heart,IDhadtheheartID
6422,not a pretty sight,IDnotaprettysightID
23473,crystal clear,IDcrystalclearID
18487,with better will,IDwithbetterwillID
30113,getting the better of,IDgettingthebetterofID
27605,over the moon,IDoverthemoonID
32288,As luck would have it,IDasluckwouldhaveitID


In [63]:
# Save the list of idioms
option_2_idiom_token_file = output_token_dir + 'option2_idiom_tokens.txt'
unique_tokens = df_data['idiom_phrase_token'].unique()
np.savetxt(option_2_idiom_token_file, unique_tokens, fmt='%s', header='')
print(f'Saved the {unique_tokens.shape[0]} tokens to {option_2_idiom_token_file}')

Saved the 10871 tokens to ../data/token_files/option2_idiom_tokens.txt
